In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yaml
from keras.datasets import fashion_mnist
import wandb

from utils.neural_network import NeuralNetwork
from utils.wandb_callback import WandbCallback
from utils.helper_functions import get_optimizer, create_validation_set

In [2]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bullseye2608 (bullseye2608-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
(X_train, X_val, y_train, y_val) = create_validation_set(X_train, y_train, val_ratio=0.1, seed=42)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

X_train_flat = X_train.reshape(X_train.shape[0], -1) / 255.0
X_val_flat = X_val.reshape(X_val.shape[0], -1) / 255.0
X_test_flat = X_test.reshape(X_test.shape[0], -1) / 255.0

### Trials

In [9]:
nn = NeuralNetwork(layer_sizes=[784, 128, 128, 128, 10], 
                   activation_functions=['relu', 'relu', 'relu', 'softmax'], 
                   weight_init='xavier',
                   weight_decay=0.0, LOG_EACH=True)

H, A = nn.forward_propagation(X_train_flat)
loss = nn.compute_loss(H[-1], y_train)
print(f'{nn.compute_accuracy(X_val_flat, y_val) :>.6f}')

nn.set_optimizer({'name':'nadam', 'learning_rate':0.001, 'epsilon':1e-7, 'beta1':0.95})

0.084667


In [10]:
num_trial_datapoints = 54000

LOG_EACH = True

nn.train(X_train_flat[:num_trial_datapoints], 
         y_train[:num_trial_datapoints], 
         X_val_flat, y_val, 
         batch_size=64, 
         num_epochs=10, 
         loss_type='cross_entropy', 
         log_every=5000)

LOG_EACH = False

print('--'*20,'DONE','--'*20)
print(nn.compute_accuracy(X_test_flat, y_test))

Running NadamOptimizer self.learning_rate = 0.001 self.beta1 = 0.95 self.beta2 = 0.999 self.epsilon = 1e-07
Epoch  1/10, Iteration   0/844 --> Train Loss: 2.35104, Val Loss: 2.16716
Epoch  6/10, Iteration 780/844 --> Train Loss: 0.19660, Val Loss: 0.31810
---------------------------------------- DONE ----------------------------------------
0.8774


### WANDB SWEEP

In [13]:
def wandb_sweep_helper_function(config=None):
    with wandb.init(
        entity="bullseye2608-indian-institute-of-technology-madras",
        project="fashion_mnist_hp_search",
        config=config):
        
        config = wandb.config
        
        
        layer_sizes = [784] + [config.hidden_size]*config.hidden_layers + [10]
        activation_functions = [config.activation]*config.hidden_layers + ['softmax']
        
        nn = NeuralNetwork(layer_sizes=layer_sizes, 
                           activation_functions=activation_functions,
                           weight_init=config.weight_init, 
                           weight_decay=config.weight_decay)
        
        wandb_callback = WandbCallback()
        
        optimizer = get_optimizer(config.optimizer, config.learning_rate)
        nn.set_optimizer(optimizer)
        
        history = nn.train(
            X_train_flat, 
            y_train, 
            X_val_flat, 
            y_val, 
            batch_size=config.batch_size, 
            num_epochs=config.epochs, 
            loss_type='cross_entropy', 
            log_every=1000,
            callback=wandb_callback  # Assuming your NeuralNetwork class supports callbacks
        )
        
        test_accuracy = nn.compute_accuracy(X_test_flat, y_test)
        wandb.log({"test_accuracy": test_accuracy})
        print(f"Test Accuracy: {test_accuracy:.4f}")
        
        return history


with open("sweep_config.yaml", "r") as file:
    sweep_config = yaml.safe_load(file)


In [9]:
# # Create New sweep

# sweep_id = wandb.sweep(sweep_config, 
#                        entity="bullseye2608-indian-institute-of-technology-madras",
#                        project="fashion_mnist_hp_search")

# # Run the sweep
# wandb.agent(sweep_id, wandb_sweep_helper_function, count=60)

In [ ]:
# Continue the sweep

sweep_id_cont = "bullseye2608-indian-institute-of-technology-madras/fashion_mnist_hp_search/vhbqpquu"

wandb.agent(sweep_id_cont, wandb_sweep_helper_function, count=150)

wandb: Agent Starting Run: xhj05fg2 with config:
wandb: 	activation: relu
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	hidden_layers: 4
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: nadam
wandb: 	weight_decay: 0.0005
wandb: 	weight_init: xavier


Test Accuracy: 0.8662
